# Доверительные интервалы
### Вопрос 1
Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X∼N(μ,σ^{2})$ лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки.

In [1]:
alpha = 1 - 0.997

In [3]:
from scipy.stats import norm

In [5]:
round(norm.ppf(1 - alpha/2), 4)

2.9677

### Вопрос 2

Пусть $X∼N(μ,σ^{2})$. Какое распределение имеет величина $\frac{X_{n}−μ}{\frac{S_{n}}{\sqrt{n}}}$?

**Ответ**: $St(n-1)$

### Вопрос 3
Выберите все распределения с несимметричной функцией плотности
* ~~Гаусса~~
* хи-квадрат
* Фишера
* ~~Стьюдента~~

### Вопрос 4
Какое из выражений задаёт доверительный интервал для разности долей в связанных выборках?

**Ответ**: $$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

### Вопрос 5
В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [123]:
s1 = 104
s2 = 189

n1 = 11037
n2 = 11034

In [124]:
p1 = float(s1)/n1
p2 = float(s2)/n2

round(p1 - p2, 4)

-0.0077

### Вопрос 6
Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.
$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [125]:
z = norm.ppf(0.975)
delta = z*(p1*(1-p1)/n1 + p2*(1-p2)/n2)**0.5

In [126]:
(p1 - p2 - delta, p1 - p2 + delta)

(-0.010724297276960124, -0.0046877506750494392)

### Вопрос 7

Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину p/1−p, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить $\hat{p}$. 

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [127]:
odds1 = p1/(1-p1)
odds2 = p2/(1-p2)
odds1, odds2

(0.009512485136741973, 0.017427385892116183)

In [128]:
round(odds2/odds1, 4)

1.8321

### Вопрос 8

Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:
 * составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
 * установите random seed=0;
 * сделайте по 1000 псевдовыборок из каждой группы пациентов.

In [129]:
placebo = np.array(s2*[1] + (n2 - s2)*[0])
aspirin = np.array(s1*[1] + (n1 - s1)*[0])

In [130]:
import numpy as np
np.random.seed(0)

In [131]:
def get_bootstrap_samples(data, n_samples):
    np.random.seed(0)
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha = 0.05):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

def get_odds_values(values):
    p = float(sum(values))/len(values)
    return p/(1-p)

In [132]:
placebo_odds = map(get_odds_values, get_bootstrap_samples(placebo, 1000))
aspirin_odds = map(get_odds_values, get_bootstrap_samples(aspirin, 1000))

In [133]:
stat_intervals(map(lambda x: x[0]/x[1], zip(placebo_odds, aspirin_odds)))

array([ 1.63035462,  2.10397776])